In [1]:
from AC_modules.Constructor import *
from Utils import A3C_training
from importlib import reload

In [2]:
# Variable parameters
X = 5
Y = 5
initial = [0,0]
goal = [4,4]
MAX_STEPS = 100

game_params = dict(x=X, y=Y, initial=initial, goal=goal, max_steps=MAX_STEPS, 
                   greyscale_state=True, return_ohe=True)

In [3]:
relational_HPs = torch.load("Results/Sandbox/Supervised/best_HP_S_chosen-residual_UMUT")
relational_HPs['n_features'] = 16 # reduce because otherwise it crashes
relational_HPs

{'n_kernels': 36,
 'n_features': 16,
 'n_heads': 1,
 'n_attn_modules': 2,
 'feature_hidden_dim': 16,
 'feature_n_residuals': 1}

In [4]:
name = "OheNet"
action_space = 4
#n_features = 16

#HPs = dict(gamma=0.99, tau=0.3, n_steps=5, H=1e-3, **relational_HPs)
HPs = dict(gamma=0.99, tau=0.3, n_steps=5, H=1e-3, n_features=32, map_size=X)
learning_rate = 1e-4

if torch.cuda.is_available():
    HPs['device'] = 'cuda'
else:
    HPs['device'] = 'cpu'
    
print("Using device "+HPs['device'])
agent_constructor = ActorCriticConstructor(name, True, action_space, **HPs)

Using device cpu
Model:  <class 'AC_modules.Networks.OheNet'>
self.model:  <class 'AC_modules.Networks.OheNet'>
self.shared:  True
self.args:  (4,)
self.kwargs:  {'gamma': 0.99, 'tau': 0.3, 'n_steps': 5, 'H': 0.001, 'n_features': 32, 'map_size': 5, 'device': 'cpu'}


In [17]:
reload(A3C_training)

<module 'Utils.A3C_training' from '/home/nicola/Nicola_unipd/MasterThesis/RelationalDeepRL/Utils/A3C_training.py'>

In [ ]:
results = A3C_training.train_sandbox(agent_constructor, learning_rate, game_params, n_training_threads=3, n_episodes=10,
                  max_steps=MAX_STEPS, return_agent=True, random_init=True)

device:  cpu
device:  cpu
Test process started
Process 1 started
Process 2 started
Process 3 started
All processes started


In [14]:
import torch.multiprocessing as mp

In [16]:
help(mp.Queue().put )

Help on method put in module multiprocessing.queues:

put(obj, block=True, timeout=None) method of multiprocessing.queues.Queue instance

